In [0]:
#Import neccessary inbuilt functions
import numpy as np
import pandas as pd
import math
import time
from itertools import combinations

# Code to read csv file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:

link = 'https://drive.google.com/open?id=1rA4YcPrXc43YnlSRbnvxcSzz9ZNbJMWx' # The shareable link
fluff, id = link.split('=')
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('bank.csv')  

# from google.colab import files
# uploaded = files.upload()

In [42]:
#Explore Dataset
df = pd.read_csv('bank.csv', header=0, sep=';')
#4521 rows and 17 columns
df.shape

(4521, 17)

In [43]:
rows=4521
title = list(df.columns.values)
features = title[:-1]
#Dependent variable to predict Independent value Y
print(features)

['age', 'job', 'marital', 'education', 'default', 'balance', 'housing', 'loan', 'contact', 'day', 'month', 'duration', 'campaign', 'pdays', 'previous', 'poutcome']


In [12]:
#change month string to int
import calendar
d = dict((v.lower(),k) for k,v in enumerate(calendar.month_abbr))
df.month = df.month.map(d)
print(df.head(10))


   age            job  marital  education  ... pdays  previous poutcome   y
0   30     unemployed  married    primary  ...    -1         0  unknown  no
1   33       services  married  secondary  ...   339         4  failure  no
2   35     management   single   tertiary  ...   330         1  failure  no
3   30     management  married   tertiary  ...    -1         0  unknown  no
4   59    blue-collar  married  secondary  ...    -1         0  unknown  no
5   35     management   single   tertiary  ...   176         3  failure  no
6   36  self-employed  married   tertiary  ...   330         2    other  no
7   39     technician  married  secondary  ...    -1         0  unknown  no
8   41   entrepreneur  married   tertiary  ...    -1         0  unknown  no
9   43       services  married    primary  ...   147         2  failure  no

[10 rows x 17 columns]


In [41]:
# unique values for each feature to use in VFDT
feature_values = {f:None for f in features}
for f in features:
    feature_values[f] = df[f].unique()

print(feature_values)

{'age': array([30, 33, 35, 59, 36, 39, 41, 43, 20, 31, 40, 56, 37, 25, 38, 42, 44,
       26, 55, 67, 53, 68, 32, 49, 78, 23, 52, 34, 61, 45, 48, 57, 54, 63,
       51, 29, 50, 27, 60, 28, 21, 58, 22, 46, 24, 77, 75, 47, 70, 65, 64,
       62, 66, 19, 81, 83, 80, 71, 72, 69, 79, 73, 86, 74, 76, 87, 84]), 'job': array(['unemployed', 'services', 'management', 'blue-collar',
       'self-employed', 'technician', 'entrepreneur', 'admin.', 'student',
       'housemaid', 'retired', 'unknown'], dtype=object), 'marital': array(['married', 'single', 'divorced'], dtype=object), 'education': array(['primary', 'secondary', 'tertiary', 'unknown'], dtype=object), 'default': array(['no', 'yes'], dtype=object), 'balance': array([ 1787,  4789,  1350, ...,  -333, -3313,  1137]), 'housing': array(['no', 'yes'], dtype=object), 'loan': array(['no', 'yes'], dtype=object), 'contact': array(['cellular', 'unknown', 'telephone'], dtype=object), 'day': array([19, 11, 16,  3,  5, 23, 14,  6, 17, 20, 13, 30, 29, 2

In [29]:
# convert df to data examples
array = df.head(4000).values
set1 = []
set2 = []
set3 = []
set4 = []
possible_split_features = title[:-1]
count = 0
for i in range(len(array)):
    count += 1
    if (count <= 1000):
        set1.append(array[i])
    elif (count > 1000 and count <= 2000):
        set2.append(array[i])
    elif (count > 2000 and count <= 3000):
        set3.append(array[i])    
    else:
        set4.append(array[i])
# to simulate continous training, modify the tree for each training set
examples = [set1, set2, set3, set4]

print(set1[:5])

[array([30, 'unemployed', 'married', 'primary', 'no', 1787, 'no', 'no',
       'cellular', 19, 10, 79, 1, -1, 0, 'unknown', 'no'], dtype=object), array([33, 'services', 'married', 'secondary', 'no', 4789, 'yes', 'yes',
       'cellular', 11, 5, 220, 1, 339, 4, 'failure', 'no'], dtype=object), array([35, 'management', 'single', 'tertiary', 'no', 1350, 'yes', 'no',
       'cellular', 16, 4, 185, 1, 330, 1, 'failure', 'no'], dtype=object), array([30, 'management', 'married', 'tertiary', 'no', 1476, 'yes', 'yes',
       'unknown', 3, 6, 199, 4, -1, 0, 'unknown', 'no'], dtype=object), array([59, 'blue-collar', 'married', 'secondary', 'no', 0, 'yes', 'no',
       'unknown', 5, 5, 226, 1, -1, 0, 'unknown', 'no'], dtype=object)]


In [0]:
# test set is different from training set
n_test = 500
test_set = df.tail(n_test).values
test = []
for i in range(len(test_set)):
    test.append(test_set[i])

In [0]:
#Codes are copied but changed some part of codes to fit my bank Dataset 
#Source and references:
#https://homes.cs.washington.edu/~pedrod/papers/kdd00.pdf
#http://huawei-noah.github.io/streamDM/docs/HDT.html
#https://github.com/doubleplusplus/incremental_decision_tree-CART-Random_Forest_python/blob/master/vfdt.py

# VFDT node class
class vfdt_node:
    
    def __init__(self, possible_split_features):
        self.parent = None
        self.left_child = None
        self.right_child = None
        self.split_feature = None
        self.split_value = None
        self.new_examples_seen = 0
        self.total_examples_seen = 0
        self.class_frequency = {}
        self.nijk = {i:{} for i in possible_split_features}
        self.possible_split_features = possible_split_features

    def add_children(self, split_feature, left, right):
        self.split_feature = split_feature
        self.left_child = left
        self.right_child = right
        left.parent = self
        right.parent = self
        self.nijk.clear()  # reset stats

    # recursively trace down the tree to distribute data examples to corresponding leaves
    def sort_example(self, example):
        if (not self.is_leaf()):
            index = self.possible_split_features.index(self.split_feature)
            value = example[:-1][index]

            try:  # continous value
                if value <= self.split_value:
                    return self.left_child.sort_example(example)
                else:
                    return self.right_child.sort_example(example)
            except TypeError:  # discrete value
                if value in self.split_value[0]:
                    return self.left_child.sort_example(example)
                else:
                    return self.right_child.sort_example(example)
        else:
            return(self)

    def is_leaf(self):
        return(self.left_child == None and self.right_child == None)

    # the most frequent classification
    def most_frequent(self):
        if (self.class_frequency):
            prediction = max(self.class_frequency, key=self.class_frequency.get)
        else:
            # if self.class_frequency dict is empty, go back to parent
            class_frequency = self.parent.class_frequency
            prediction = max(class_frequency, key=class_frequency.get)
        return(prediction)

    # upadate leaf stats in order to calculate infomation gain
    def update_stats(self, example):
        label = example[-1]
        feats = self.possible_split_features
        for i in feats:
            if (i != None):
                value = example[:-1][feats.index(i)]
                if (value not in self.nijk[i]):
                    d = {label : 1}
                    self.nijk[i][value] = d
                else:
                    if (label not in self.nijk[i][value]):
                        self.nijk[i][value][label] = 1
                    else:
                        self.nijk[i][value][label] += 1
        self.total_examples_seen += 1
        self.new_examples_seen += 1
        if (label not in self.class_frequency):
            self.class_frequency[label] = 1
        else:
            self.class_frequency[label] += 1

    def check_not_splitting(self):
        # compute Gini index for not splitting
        X0 = 1
        class_frequency = self.class_frequency
        n = sum(class_frequency.values())
        for j, k in class_frequency.items():
            X0 -= (k/n)**2
        return X0

    # use hoeffding tree model to test node split, return the split feature
    def splittable(self, delta, nmin, tau):
        if(self.new_examples_seen < nmin):
            return(None)
        else:
            self.new_examples_seen = 0  # reset
        nijk = self.nijk
        min = 1
        second_min = 1
        Xa = ''
        split_value = None
        for feature in self.possible_split_features:
            if (len(nijk[feature]) != 1):
                gini, value = self.Gini(feature)
                if (gini < min):
                    min = gini
                    Xa = feature
                    split_value = value
                elif (min < gini < second_min):
                    second_min = gini
            else:
                return None

        sigma = self.hoeffding_bound(delta)
        X0 = self.check_not_splitting()
        if min < X0:
            if (second_min - min > sigma):
                return [Xa, split_value]
            elif (sigma < tau and second_min - min < tau):
                return [Xa, split_value]
            else:
                return None
        else:
            return None

    def hoeffding_bound(self, delta):
        n = self.total_examples_seen
        R = np.log(len(self.class_frequency))
        return (R * R * np.log(1/delta) / (2 * n))**0.5

    def Gini(self, feature):
        '''
        Gini(D) = 1 - Sum(pi^2)
        Gini(D, F=f) = |D1|/|D|*Gini(D1) + |D2|/|D|*Gini(D2)
        '''
        njk = self.nijk[feature]
        D = self.total_examples_seen
        class_frequency = self.class_frequency

        m1 = 1  # minimum gini
        m2 = 1  # second minimum gini
        Xa_value = None
        test = next(iter(njk))  # test j value
        # if not isinstance(test, np.object):
        try:  # continous feature values
            test += 0
            sort = sorted([j for j in njk.keys()])
            split = []
            for i in range(1, len(sort)):
                temp = (sort[i-1] + sort[i])/2
                split.append(temp)

            D1 = 0
            D1_class_frequency = {j:0 for j in class_frequency.keys()}
            for index in range(len(split)):
                nk = njk[sort[index]]

                for j in nk:
                    D1_class_frequency[j] += nk[j]
                D1 += sum(nk.values())
                D2 = D - D1
                g_d1 = 1
                g_d2 = 1

                D2_class_frequency = {}
                for key, value in class_frequency.items():
                    if key in D1_class_frequency:
                        D2_class_frequency[key] = value - D1_class_frequency[key]
                    else:
                        D2_class_frequency[key] = value

                for key, v in D1_class_frequency.items():
                    g_d1 -= (v/float(D1))**2
                for key, v in D2_class_frequency.items():
                    g_d2 -= (v/float(D2))**2
                g = g_d1*D1/D + g_d2*D2/D
                if g < m1:
                    m1 = g
                    Xa_value = split[index]
                elif m1 < g < m2:
                    m2 = g
            return [m1, Xa_value]

        # discrete feature_values
        except TypeError:
            length = len(njk)
            feature_values = list(njk.keys())
            right = None
            if length > 10:  # too many discrete feature values
                for j, k in njk.items():
                    D1 = sum(k.values())
                    D2 = D - D1
                    g_d1 = 1
                    g_d2 = 1

                    D2_class_frequency = {}
                    for key, value in class_frequency.items():
                        if key in k:
                            D2_class_frequency[key] = value - k[key]
                        else:
                            D2_class_frequency[key] = value

                    for key, v in k.items():
                        g_d1 -= (v/D1)**2

                    if D2 != 0:
                        for key, v in D2_class_frequency.items():
                            g_d2 -= (v/D2)**2
                    g = g_d1*D1/D + g_d2*D2/D
                    if g < m1:
                        m1 = g
                        Xa_value = j
                    elif m1 < g < m2:
                        m2 = g
                right = list(np.setdiff1d(feature_values, Xa_value))

            else:  # fewer discrete feature values
                comb = self.select_combinations(feature_values)
                for i in comb:
                    left = list(i)
                    D1 = 0
                    D2 = 0
                    D1_class_frequency = {key:0 for key in class_frequency.keys()}
                    D2_class_frequency = {key:0 for key in class_frequency.keys()}
                    for j,k in njk.items():
                        for key, value in class_frequency.items():
                            if j in left:
                                if key in k:
                                    D1 += sum(k.values())
                                    D1_class_frequency[key] += k[key]
                            else:
                                if key in k:
                                    D2 += sum(k.values())
                                    D2_class_frequency[key] += k[key]
                    g_d1 = 1
                    g_d2 = 1
                    for key, v in D1_class_frequency.items():
                        g_d1 -= (v/D1)**2
                    for key, v in D2_class_frequency.items():
                        g_d2 -= (v/D1)**2
                    g = g_d1*D1/D + g_d2*D2/D
                    if g < m1:
                        m1 = g
                        Xa_value = left
                    elif m1 < g < m2:
                        m2 = g
                right = list(np.setdiff1d(feature_values, Xa_value))
            return [m1, [Xa_value, right]]

    def select_combinations(self, feature_values):
        combination = []
        e = len(feature_values)
        if e % 2 == 0:
            end = int(e/2)
            for i in range(1, end+1):
                if i == end:
                    cmb = list(combinations(feature_values, i))
                    enough = int(len(cmb)/2)
                    combination.extend(cmb[:enough])
                else:
                    combination.extend(combinations(feature_values, i))
        else:
            end = int((e-1)/2)
            for i in range(1, end+1):
                combination.extend(combinations(feature_values, i))

        return combination

In [0]:
# very fast decision tree class, i.e. hoeffding tree
class vfdt:
    # parameters
    # feature_values  # number of values of each feature # dict
    # feature_values = {feature:[unique values list]}
    # delta   # used for hoeffding bound
    # tau  # used to deal with ties
    # nmin  # used to limit the G computations

    def __init__(self, feature_values, delta=0.05, nmin=50, tau=0.1):
        self.feature_values = feature_values
        self.delta = delta
        self.nmin = nmin
        self.tau = tau
        features = list(feature_values.keys())
        self.root = vfdt_node(features)
        self.n_examples_processed = 0

    # update the tree by adding training example
    def update(self, example):
        self.n_examples_processed += 1
        node = self.root.sort_example(example)
        node.update_stats(example)

        result = node.splittable(self.delta, self.nmin, self.tau)
        if result != None:
            feature = result[0]
            value = result[1]
            self.node_split(node, feature)
            node.split_value = value

    # split node, produce children
    def node_split(self, node, split_feature):
        features = node.possible_split_features
        # replace deleted split feature with None
        # new_features = [None if f == split_feature else f for f in features]
        left = vfdt_node(features)
        right = vfdt_node(features)
        node.add_children(split_feature, left, right)

    # predict test example's classification
    def predict(self, example):
        leaf = self.root.sort_example(example)
        prediction = leaf.most_frequent()
        return(prediction)

    # accuracy of a test set
    def accuracy(self, examples):
        correct = 0
        for ex in examples:
            if (self.predict(ex) == ex[-1]):
                correct +=1
        return(float(correct) / len(examples))

    def print_tree(self, node):
        if node.is_leaf():
            print('Leaf')
        else:
            print(node.split_feature)
            self.print_tree(node.left_child)
            self.print_tree(node.right_child)



In [33]:
# heoffding bound parameter delta: with 1 - delta probability
# the true mean is at least r - gamma
# vfdt parameter nmin: test split if new sample size > nmin
start_time = time.time()
tree = vfdt(feature_values, delta=0.05, nmin=100, tau=0.05)
print('Total data size: ', rows,'\n')
print('Test set (tail): ', len(test_set),'\n')
n = 0
for training_set in examples:
    n += len(training_set)
    for ex in training_set:
        tree.update(ex)
    print('Training set:', n, end=', ')
    print('ACCURACY: %.4f \n' % tree.accuracy(test))

#tree.print_tree(tree.root)
print("--- Running time: %.6f seconds ---" % (time.time() - start_time))


Total data size:  4521 

Test set (tail):  500 

Training set: 1000, ACCURACY: 0.8840 

Training set: 2000, ACCURACY: 0.8840 

Training set: 3000, ACCURACY: 0.8840 

Training set: 4000, ACCURACY: 0.8860 

--- Running time: 0.287230 seconds ---
